In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder,normalize
from sklearn.decomposition import PCA, NMF
import data_processing as dp
from scipy import signal
from scipy.stats import skew,pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations
from sklearn.feature_selection import SelectKBest,f_classif,chi2,mutual_info_classif,VarianceThreshold,RFE,SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from scipy.stats import pearsonr

In [30]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

.\data\file_name.txt
.\data\G04_FoG_trial_1_emg.csv
.\data\G04_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_1_emg.csv
.\data\G06_FoG_trial_2_emg.csv
.\data\G06_FoG_trial_3_emg.csv
.\data\G07_Freezing_Trial1_trial_1_emg.csv
.\data\G08_FoG_1_trial_1_emg.csv
.\data\G08_FoG_2_trial_1_emg.csv
.\data\G11_FoG_trial_1_emg.csv
.\data\G11_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_A_FoG_trial_3_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_1_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_2_emg.csv
.\data\P379_M050_2_OFF_B_FoG_trial_3_emg.csv
.\data\P551_M050_2_A_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_1_emg.csv
.\data\P551_M050_2_B_FoG_trial_2_emg.csv
.\data\P812_M050_2_B_FoG_trial_1_emg.csv
.\data\P812_M050_2_B_FoG_trial_2_emg.csv
.\data\some_of_rest.txt
.\data\其他\labels.txt
.\data\其他\P812_M050_2_B_FoG_trial1_annotation.csv
.\data\其他\P812_M050_2_B_FoG_trials.mat
.\data\其他\P812_M050_2_B_FoG_trial_1_out_left

In [ ]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [33]:
Data = pd.read_csv('./processed data/dataframe_W1024_S256_DWTLmax_samelabel_sc_detrend.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'G04_FoG_trial_1_emg.csv'
drop2= 'G04_FoG_trial_2_emg.csv'
drop3= 'P812_M050_2_B_FoG_trial_1_emg.csv'
ind_drop = (df.columns!=drop) & (df.columns!=drop2) #& (df.columns!=drop3)
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns),np.array(df3.loc[:,0])])
ind = Data.File.isin('detrend_300_'+files)
ind2 = Data.File == drop
#ind = (Data.File != drop) & (Data.File != drop2)
Data_sel = Data#[ind]
Data_rest = Data[ind2]
#ind2 = Data_rest.File == drop
#Data_rest = Data_rest[ind2]
#Data_rest

In [5]:
feature_col = ['_IEMG','_MAV','_SSI','_VAR','_RMS',
               '_WL','_ZC','_SSC','_WAMP','_skew','_Acti','_AR','_HIST','_MDF']

feature_all = Data_sel.iloc[:,1:-1]
#ind_temp1 = feature_all.columns.str.contains('_mDWT')
#ind_temp2 = feature_all.columns.str.contains('_Coe')
#ind_temp3 = feature_all.columns.str.contains('_Scale')
#ind_temp4 = feature_all.columns.str.contains('_HIST')
#ind_temp = ind_temp1|ind_temp2|ind_temp3|ind_temp4
ind_temp = feature_all.columns.str.contains('_Acti')
#feature = feature_all.loc[:,~ind_temp]
feature = Data_sel.iloc[:,1:-1]
#feature = Data_sel.iloc[:,:-2]
y = Data_sel.Label
feature2_all = Data_rest.iloc[:,1:-1]
#feature2 = feature2_all.loc[:,~ind_temp]
feature2 = Data_rest.iloc[:,1:-1]
#feature2 = Data_rest.iloc[:,:-2]
y2 = Data_rest.Label

In [36]:
feature = pd.DataFrame()
feature2 = pd.DataFrame()
y = pd.DataFrame()
y2 = pd.DataFrame()
m = 0
for i in 'detrend_300_'+df.columns:
    ind = Data_sel.File==i
    temp = Data_sel[ind]
    #if i != 'G08_FoG_2_trial_1_emg.csv':
        #feature = pd.concat([feature,temp.iloc[:,1:-1]])
        #y = pd.concat([y,temp.iloc[:,0]])
    #else:
    for j in set(temp.Label):
        ind2 = temp.Label == j
        temp2 = temp[ind2]
        l = len(temp2)
        feature = pd.concat([feature,temp2.iloc[:int(0.8*l),1:-1]])
        y = pd.concat([y,temp2.iloc[:int(0.8*l),0]])
        feature2 = pd.concat([feature2,temp2.iloc[int(0.8*l):,1:-1]])
        y2 = pd.concat([y2,temp2.iloc[int(0.8*l):,0]])
y = np.array(y)[:,0]
y2 = np.array(y2)[:,0]
#ind_temp = feature.columns.str.contains('_mDWT')
#feature = feature.loc[:,~ind_temp]
#feature2 = feature2.loc[:,~ind_temp]

In [38]:
#ind_temp1 = feature_all.columns.str.contains('_MF')
#ind_temp2 = feature_all.columns.str.contains('_MDF')
#ind_temp3 = feature_all.columns.str.contains('_AR')
#ind_temp4 = feature_all.columns.str.contains('_MNF')
#ind_temp = ind_temp1|ind_temp2|ind_temp3|ind_temp4
#feature = feature.loc[:,~ind_temp]
#feature2 = feature2.loc[:,~ind_temp]
#('detrend_300_'+files)
y2.shape

(649,)

In [35]:
Data_sel

,Label,LEFT_TA_IEMG,LEFT_TS_IEMG,LEFT_BF_IEMG,LEFT_RF_IEMG,RIGHT_TA_IEMG,RIGHT_TS_IEMG,RIGHT_BF_IEMG,RIGHT_RF_IEMG,LEFT_TA_SSI,...,RIGHT_RF_mDWT1,RIGHT_RF_mDWT2,RIGHT_RF_mDWT3,RIGHT_RF_mDWT4,RIGHT_RF_mDWT5,RIGHT_RF_mDWT6,RIGHT_RF_mDWT7,RIGHT_RF_mDWT8,RIGHT_RF_mDWT9,File
0,0,532.99800,682.87820,617.53705,496.16614,448.80520,706.48083,752.47130,588.93210,1024.0030,...,285.226318,154.851379,59.555408,31.748150,3.224998,1.554702,0.766442,0.344387,0.195914,detrend_300_G06_FoG_trial_1_emg.csv
1,0,610.86285,688.22974,597.88810,566.05260,516.76044,660.64325,485.37173,482.71650,1024.0051,...,331.483459,208.414215,122.168480,97.462158,77.872330,57.111313,28.516342,14.124079,6.673710,detrend_300_G06_FoG_trial_1_emg.csv
2,0,717.85050,731.97920,594.86550,674.61707,651.91720,717.54380,542.46875,607.74850,1024.0077,...,291.858368,109.412689,42.897900,23.493301,2.436239,1.730924,0.861943,0.425198,0.203946,detrend_300_G06_FoG_trial_1_emg.csv
3,0,784.56880,767.33844,636.39390,761.74896,743.55646,753.26245,582.45764,707.13880,1024.0006,...,345.837891,156.745499,52.142414,29.755404,5.823022,4.090141,2.037884,1.007435,0.474716,detrend_300_G06_FoG_trial_1_emg.csv
4,0,784.83930,761.79780,694.74960,790.04110,730.96857,763.32684,623.00880,793.52216,1023.9998,...,385.259094,196.491730,57.551464,25.937048,14.986317,10.811022,5.363523,2.578475,1.295517,detrend_300_G06_FoG_trial_1_emg.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,0,742.91990,697.15643,622.27050,819.97250,718.64650,567.61755,793.67810,672.50260,1024.0135,...,350.094971,181.753113,64.880173,35.884857,24.737333,18.071045,9.008681,4.517670,2.074450,detrend_300_P812_M050_2_B_FoG_trial_2_emg.csv
3164,0,717.32360,697.38390,581.20807,813.19196,734.41210,680.13920,787.93365,643.52880,1024.0034,...,315.500580,167.535095,45.436825,19.283266,16.084208,11.476061,5.672655,2.823398,1.397688,detrend_300_P812_M050_2_B_FoG_trial_2_emg.csv
3165,0,643.79970,676.41580,580.67676,812.15770,720.61710,659.99330,786.36615,675.54670,1024.0194,...,412.005890,259.395691,151.239639,113.130943,101.270905,74.047607,36.972191,18.248516,8.697049,detrend_300_P812_M050_2_B_FoG_trial_2_emg.csv
3166,0,611.56714,689.03000,609.06960,814.09170,726.28200,661.06270,785.89870,665.97300,1024.0010,...,330.379303,187.993164,67.779640,25.933273,3.889570,2.537868,1.293690,0.790001,0.241916,detrend_300_P812_M050_2_B_FoG_trial_2_emg.csv


In [180]:
from scipy.stats import entropy
def get_features_from_dwt(data,wavelet='db7',level=5):
    coes = pywt.wavedec(data,wavelet=wavelet,mode=0,level=level,axis=1)
    #columns = pd.Index(['LEFT_TA', 'LEFT_TS', 'LEFT_BF', 'LEFT_RF',
    #   'RIGHT_TA', 'RIGHT_TS', 'RIGHT_BF', 'RIGHT_RF'])
    columns = pd.Index(['LEFT_TA', 'LEFT_TS','RIGHT_TA', 'RIGHT_TS'])
    feature = pd.DataFrame()
    for i in range(len(coes)):
        #IEMG = pd.DataFrame(compute_IEMG(coes[i]),columns=columns+'_IEMG')
        RMS = pd.DataFrame(dp.compute_RMS(coes[i]),columns=columns+'_RMS%d'%i)
        #WL = pd.DataFrame(dp.compute_WL(coes[i]),columns=columns+'_WL%d'%i)
        ZC = pd.DataFrame(dp.compute_ZC(coes[i],1e-6),columns=columns+'_ZC%d'%i)
        ku = pd.DataFrame(dp.compute_ku(coes[i]),columns=columns+'_ku%d'%i)
        #SSC = pd.DataFrame(dp.compute_SSC(coes[i],threshold_SSC),columns=columns+'_SSC%d'%i)
        #WAMP = pd.DataFrame(dp.compute_WAMP(coes[i],threshold_WAMP),columns=columns+'_WAMP%d'%i)
        skew = pd.DataFrame(dp.compute_Skewness(coes[i]),columns=columns+'_skew%d'%i)
        Acti = pd.DataFrame(dp.compute_Acti(coes[i]),columns=columns+'_Acti%d'%i)
        #AR = pd.DataFrame(dp.compute_AR(coes[i]),columns=columns+'_AR%d'%i)
        #AR = dp.compute_AR_pd(coes[i])
        #HIST = dp.compute_HIST_pd(coes[i],bins=bins,ranges=ranges)
        #FHIST = dp.compute_FHIST_pd(coes[i],bins=fbins,ranges=franges,threshold=threshold_F)
        #MF = dp.compute_MaxFreq_pd(coes[i],num=num)
        #MDF = pd.DataFrame(dp.compute_MDF(coes[i]),columns=columns+'_MDF%d'%i)
        #MNF = pd.DataFrame(dp.compute_MNF(coes[i]),columns=columns+'_MNF%d'%i)
        feature = pd.concat([feature,RMS,ZC,ku,skew,Acti],axis =1)
    return feature

def get_features_from_dwt2(data,wavelet='db7',level=5):
    coes = pywt.wavedec(data,wavelet=wavelet,mode=0,level=level,axis=1)
    #columns = pd.Index(['LEFT_TA', 'LEFT_TS', 'LEFT_BF', 'LEFT_RF',
    #   'RIGHT_TA', 'RIGHT_TS', 'RIGHT_BF', 'RIGHT_RF'])
    columns = pd.Index(['LEFT_TA', 'LEFT_TS','RIGHT_TA', 'RIGHT_TS'])
    feature = pd.DataFrame()
    for i in range(len(coes)):
        MAV = pd.DataFrame(dp.compute_MAV(coes[i]),columns=columns+'_MAV%d'%i)
        RMS = pd.DataFrame(dp.compute_RMS(coes[i]),columns=columns+'_RMS%d'%i)
        #WL = pd.DataFrame(dp.compute_WL(coes[i]),columns=columns+'_WL%d'%i)
        ZC = pd.DataFrame(dp.compute_ZC(coes[i],1e-6),columns=columns+'_ZC%d'%i)
        ku = pd.DataFrame(dp.compute_ku(coes[i]),columns=columns+'_ku%d'%i)
        Mean = pd.DataFrame(np.mean(coes[i],axis=1),columns=columns+'_Mean%d'%i)
        STD = pd.DataFrame(np.std(coes[i],axis=1),columns=columns+'_STD%d'%i)
        #EN = pd.DataFrame(entropy(coes[i],axis=1),columns=columns+'_EN%d'%i)
        #SSC = pd.DataFrame(dp.compute_SSC(coes[i],threshold_SSC),columns=columns+'_SSC%d'%i)
        #WAMP = pd.DataFrame(dp.compute_WAMP(coes[i],threshold_WAMP),columns=columns+'_WAMP%d'%i)
        skew = pd.DataFrame(dp.compute_Skewness(coes[i]),columns=columns+'_skew%d'%i)
        #Acti = pd.DataFrame(dp.compute_Acti(coes[i]),columns=columns+'_Acti%d'%i)
        #AR = pd.DataFrame(dp.compute_AR(coes[i]),columns=columns+'_AR%d'%i)
        #AR = dp.compute_AR_pd(coes[i])
        #HIST = dp.compute_HIST_pd(coes[i],bins=bins,ranges=ranges)
        #FHIST = dp.compute_FHIST_pd(coes[i],bins=fbins,ranges=franges,threshold=threshold_F)
        #MF = dp.compute_MaxFreq_pd(coes[i],num=num)
        #MDF = pd.DataFrame(dp.compute_MDF(coes[i]),columns=columns+'_MDF%d'%i)
        #MNF = pd.DataFrame(dp.compute_MNF(coes[i]),columns=columns+'_MNF%d'%i)
        feature = pd.concat([feature,MAV,RMS,ZC,ku,Mean,STD,skew],axis =1)
    return feature

In [2]:
# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

In [3]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

1

In [279]:
# read the data and labels of df2 or df3
sc = StandardScaler(with_mean = True)
#sc = MinMaxScaler()
ind = df2.iloc[1].isna()
files = np.concatenate([np.array(df.columns),np.array('正常/'+df2.columns[ind])])
#files = np.array(df.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]])#[[2,5,6,7,8,10,11,12,13,16,17,18,19,20]])
N = len(files)
#sc = StandardScaler(with_mean=False)
width = 1024
stride = 512
start = time.time()
i = 0
X = []
Y = []
X2 = []
Y2 = []
X3 = []
Y3 = []
F = []
F2 = []
F3 = []
for file in files:
    i += 1
    if file.find('G04')==0:
        continue
    emg_data = pd.read_csv('./data/'+file)
    #emg_data.iloc[:,3:] = 
    emg_data = emg_data.dropna().reset_index(drop=True)
    #emg_data.iloc[:,3:]=normalize(emg_data.iloc[:,3:],axis=0)
    #emg_data.iloc[:,3:] = sc.fit_transform(emg_data.iloc[:,3:])
    #for j in (0,1,4,5):
        #ind = abs(zscore(emg_data.iloc[:,j+3]))>10
        #emg_data=emg_data.loc[~ind,:]
        #ind_p = zscore(emg_data.iloc[:,j+3])>10
        #ind_n = zscore(emg_data.iloc[:,j+3])<-10
        #emg_data.loc[ind_p,emg_data.columns[3+j]] = emg_data.loc[~ind_p,emg_data.columns[3+j]].max()
        #emg_data.loc[ind_n,emg_data.columns[3+j]] = emg_data.loc[~ind_n,emg_data.columns[3+j]].min()
    fn = 300
    wn=2*fn/1000
    fn1 = 400
    wn1 = 2*fn1/1000
    #fs = 1000.0  # Sample frequency (Hz)
    #f0 = 50  # Frequency to be removed from signal (Hz)
    #Q = 100.0  # Quality factor
    # Design notch filter
    #b1, a1 = signal.iirnotch(f0, Q, fs)
    #b, a = signal.butter(4, [wn,wn1], 'bandpass')
    b, a = signal.butter(4, [wn], 'lowpass')
    #b, a = signal.butter(4, [wn], 'highpass')
    #for j in ['LEFT_TA','LEFT_TS','LEFT_BF','LEFT_RF','RIGHT_TA','RIGHT_TS','RIGHT_BF','RIGHT_RF']:
        #emg_data.loc[:,j] = signal.filtfilt(b, a, emg_data.loc[:,j])
        #emg_data.loc[:,j] = signal.filtfilt(b1, a1, emg_data.loc[:,j])
    """if file==df.columns[5]:
        print(file)
        fs = 1000.0  # Sample frequency (Hz)
        f0 = 72  # Frequency to be removed from signal (Hz)
        Q = 50.0  # Quality factor
        # Design notch filter
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'LEFT_TA'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TA'])
        f0 = 75  # Frequency to be removed from signal (Hz)
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'LEFT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TS'])
        f0 = 13.2  # Frequency to be removed from signal (Hz)
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'RIGHT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'RIGHT_TS'])
    if file==df.columns[6]:
        print(file)
        fs = 1000.0  # Sample frequency (Hz)
        f0 = 40  # Frequency to be removed from signal (Hz)
        Q = 100.0  # Quality factor
        # Design notch filter
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'LEFT_TA'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TA'])
        f0 = 26.5  # Frequency to be removed from signal (Hz)
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'LEFT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TS'])
        f0 = 13.2  # Frequency to be removed from signal (Hz)
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'LEFT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TS'])
        f0 = 48  # Frequency to be removed from signal (Hz)
        b1, a1 = signal.iirnotch(f0, Q, fs)
        #emg_data.loc[:,'LEFT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TS'])
        f0 = 50  # Frequency to be removed from signal (Hz)
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'RIGHT_TA'] = signal.filtfilt(b1, a1, emg_data.loc[:,'RIGHT_TA'])
    if file==df.columns[7]:
        print(file)
        fs = 1000.0  # Sample frequency (Hz)
        f0 = 13.2  # Frequency to be removed from signal (Hz)
        Q = 50.0  # Quality factor
        # Design notch filter
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'LEFT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TS'])
        emg_data.loc[:,'RIGHT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'RIGHT_TS'])
        f0 = 26.5  # Frequency to be removed from signal (Hz)
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'LEFT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TS'])
        f0 = 50  # Frequency to be removed from signal (Hz)
        b1, a1 = signal.iirnotch(f0, Q, fs)
        emg_data.loc[:,'LEFT_TS'] = signal.filtfilt(b1, a1, emg_data.loc[:,'LEFT_TS'])
    """
    #emg_data.iloc[:,3:] = sc.fit_transform(emg_data.iloc[:,3:])
    #emg_data.iloc[:,3:]=normalize(emg_data.iloc[:,3:],axis=0)
    x_raw,y = dp.generate_window_slide_data_time_continue(emg_data,width=width,
                                        stride=stride,
                                        scaler=True,
                                        same_label=True)
    #x=np.abs(x)
    #x=dp.lowpass_filter(x,300)
    #x=dp.mean_smooth(x)
    shape = x_raw.shape
    x = np.zeros(shape)
    #x = x_raw
    for n in range(shape[0]):
        x[n,:,:] = dp.detrend(x_raw[n,:,:],50)
        for c in range(shape[2]):
            x[n,:,c] = signal.filtfilt(b,a,x[n,:,c])
    
    ind1 = []
    ind2 = []
    ind3 = []
    l = len(y)
    for j in set(y):
        ind = np.where(y == j)[0].tolist()
        l_t = len(ind)
        if (j==1)&((file==df.columns[5])|(file==df.columns[17])):
            continue
        if file == files[6]:
            ind3 += ind
            print('X2')
        else:
            ind1 += ind[:int(l_t*0.8)]
            ind2 += ind[int(l_t*0.8):int(l_t*1)]
            ind3 += ind[int(l_t*1):]

    l1 = len(ind1)
    l2 = len(ind2)
    l3 = len(ind3)

    fi = [file]*len(ind1)
    fi2 = [file]*len(ind2)
    fi3 = [file]*len(ind3)
    
    X += x[ind1].tolist()
    Y += y[ind1].tolist()
    
    X2 += x[ind2].tolist()
    Y2 += y[ind2].tolist()
    
    X3 += x[ind3].tolist()
    Y3 += y[ind3].tolist()
    
    F += fi
    F2 += fi2
    F3 += fi3
    print('%d/%d: '%(i,N)+file)

ind_c = [True,True,False,False,True,True,False,False]
X = np.array(X)#[:,:,ind_c]
Y = np.array(Y)
X2 = np.array(X2)#[:,:,ind_c]
Y2 = np.array(Y2)
X3 = np.array(X3)#[:,:,ind_c]
Y3 = np.array(Y3)
end = time.time()
duration = end-start
print('Duration: %f'%(duration))
player.Beep(1000,200)

3/174: G06_FoG_trial_1_emg.csv
4/174: G06_FoG_trial_2_emg.csv
5/174: G06_FoG_trial_3_emg.csv
6/174: G07_Freezing_Trial1_trial_1_emg.csv
X2
X2
X2
X2
7/174: G08_FoG_1_trial_1_emg.csv
8/174: G08_FoG_2_trial_1_emg.csv
9/174: G11_FoG_trial_1_emg.csv
10/174: G11_FoG_trial_2_emg.csv
11/174: P379_M050_2_OFF_A_FoG_trial_1_emg.csv
12/174: P379_M050_2_OFF_A_FoG_trial_2_emg.csv
13/174: P379_M050_2_OFF_A_FoG_trial_3_emg.csv
14/174: P379_M050_2_OFF_B_FoG_trial_1_emg.csv
15/174: P379_M050_2_OFF_B_FoG_trial_2_emg.csv
16/174: P379_M050_2_OFF_B_FoG_trial_3_emg.csv
17/174: P551_M050_2_A_FoG_trial_1_emg.csv
18/174: P551_M050_2_B_FoG_trial_1_emg.csv
19/174: P551_M050_2_B_FoG_trial_2_emg.csv
20/174: P812_M050_2_B_FoG_trial_1_emg.csv
21/174: P812_M050_2_B_FoG_trial_2_emg.csv
22/174: 正常/G02_Walking_trial_1_emg.csv
23/174: 正常/G03_Walking_trial_1_emg.csv
24/174: 正常/G03_Walking_trial_2_emg.csv
25/174: 正常/G05_Walking_struct_fixed_trial_1_emg.csv
26/174: 正常/G05_Walking_struct_fixed_trial_2_emg.csv
27/174: 正常/G05_W

1

In [258]:
ind1 = ((Y==1)|(Y==2)|(Y==6))
ind2 = ((Y2==1)|(Y2==2)|(Y2==6))
ind3 = ((Y3==1)|(Y3==2)|(Y3==6))

In [280]:
#width = 256
threshold_WAMP = 1
threshold_ZC = 0.00
#threshold_ZC = np.linspace(-4,4,11)
threshold_SSC = 0.001
bins=3
bound = 3
HIST_range = (-bound,bound)
level = 3

In [281]:
feature = dp.generate_feature_pd(X,threshold_WAMP=threshold_WAMP,
                              threshold_ZC=threshold_ZC,
                              threshold_SSC=threshold_SSC,
                              bins=bins,ranges=HIST_range,
                                level = level)
feature2 = dp.generate_feature_pd(X2,threshold_WAMP=threshold_WAMP,
                              threshold_ZC=threshold_ZC,
                              threshold_SSC=threshold_SSC,
                              bins=bins,ranges=HIST_range,
                                 level = level)
feature3 = dp.generate_feature_pd(X3,threshold_WAMP=threshold_WAMP,
                              threshold_ZC=threshold_ZC,
                              threshold_SSC=threshold_SSC,
                              bins=bins,ranges=HIST_range,
                                 level = level)
player.Beep(1000,200)

1

In [282]:
#scaler = MinMaxScaler((0,1))
scaler = StandardScaler(with_mean=True)
feature = scaler.fit_transform(feature)
feature2 = scaler.fit_transform(feature2)
feature3 = scaler.fit_transform(feature3)
# feature = normalize(feature)
# feature2 = normalize(feature2)
# feature3 = normalize(feature3)

In [137]:
#skb = SelectKBest(chi2, k=150)
skb = SelectKBest(mutual_info_classif, k=50)
feature_new=skb.fit_transform(feature, Y)
feature_new2=skb.transform(feature2)
feature_new3=skb.transform(feature3)

In [ ]:
vt = VarianceThreshold(threshold=0.005)
feature_new=vt.fit_transform(feature)
feature_new2=vt.transform(feature2)
feature_new3=vt.transform(feature3)

In [259]:
rfe = RFE(estimator=LogisticRegression(max_iter=10000), n_features_to_select=200)
feature_new = rfe.fit_transform(feature[ind1],Y[ind1])
feature_new2 = rfe.transform(feature2[ind2])
feature_new3 = rfe.transform(feature3[ind3])

In [ ]:
sfm = SelectFromModel(GradientBoostingClassifier(),max_features=80)
feature_new = sfm.fit_transform(feature[ind1],Y[ind1])
feature_new2 = sfm.transform(feature2[ind2])
feature_new3 = sfm.transform(feature3[ind3])

In [275]:
pca = PCA(n_components=200,copy=True)
feature_new = pca.fit_transform(feature)
feature_new2 = pca.transform(feature2)
feature_new3 = pca.transform(feature3)

In [ ]:
lda = LDA(n_components=2)
feature_new = lda.fit_transform(feature_new,Y[ind1])
feature_new2 = lda.transform(feature_new2)
feature_new3 = lda.transform(feature_new3)

In [181]:
feature = get_features_from_dwt2(X[:,:,[0,1,4,5]],level=5)
feature2 = get_features_from_dwt2(X2[:,:,[0,1,4,5]],level=5)

In [287]:
Y.shape

(6611,)

In [263]:
import imp
imp.reload(dp)

<module 'data_processing' from 'E:\\Document\\jupyter\\Master Thesis\\data_processing.py'>

In [128]:
(Data_rest.File=='P812_M050_2_B_FoG_trial_1_emg.csv').sum()

0

In [ ]:
width = 256
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [4]:
path = './processed data/data_set_after_window_W128_S64_sameLabel_rect.hdf5'
with h5py.File(path,'r') as f:
    x = f['data'][...]
    y = f['label2'][...]
#feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
#                              threshold_ZC=threshold_ZC,
#                              threshold_SSC=threshold_SSC,
#                              bins=bins,ranges=HIST_range)
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

1

In [ ]:
with h5py.File('./processed data/nfeatures_W256_S64_WAMP30.hdf5','r') as f:
    feature = f['features'][...]
    y = f['labels'][...]

In [ ]:
feature2,y2 = dp.pipeline_feature(path2,width=256,stride=64,
                                  scaler=False,
                                  threshold_WAMP=threshold_WAMP,
                                  threshold_ZC=threshold_ZC,
                                  threshold_SSC=threshold_SSC,
                                  bins=bins,ranges=HIST_range)

In [ ]:
with h5py.File('./processed data/nfeatures_rest_W256_S64.hdf5','r') as f:
    feature2 = f['features'][...]
    y2 = f['labels'][...]

In [ ]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)

# Model

In [23]:
import tensorflow as tf
from tensorflow.keras import layers,Model,callbacks,regularizers,models
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.metrics import precision_recall_curve,confusion_matrix,accuracy_score
from sklearn.preprocessing import OneHotEncoder,normalize,MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE

In [6]:
tf.executing_eagerly()

True

In [11]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==6))
        #ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y[ind].copy()
        ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-2')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    sc = StandardScaler(with_mean=True)
    #sc = MinMaxScaler()
    x_train = sc.fit_transform(x_full)
    pca = PCA(n_components=100)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = sc.transform(x_test)
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_loss', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=(x_test,y_test),
                        epochs=300,batch_size=32,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc,pca

def test_model(model,feature,y,sc,pca,binary=True):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        #ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y[ind].copy()
        ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}

    #print(y_01)
    feature=sc.transform(feature[ind])
    #feature = feature[ind]
    #feature=pca.transform(feature)
    y_pred=model.predict(feature)
    test = metrics.accuracy_score(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1))
    #test = metrics.accuracy_score(y_01,y_pred>0.5)
    
    print('acc:%f'%test)
    print(metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1)))
    #print(metrics.confusion_matrix(y_01,y_pred>0.5))
    return test

def sparse_cost_sensitive_loss (y_true,y_pred):
    #cost_matrix = tf.constant([[0,1.,1,1.],
    #              [2,0,5,5],
    #              [1,1,0,1],
    #              [1.,2.,1,0]])
    cost_matrix = tf.constant([[0,2.,2],
                  [1,0,1],
                  [1.0,1.,0]])
    #cost_matrix = tf.constant([[0,1.],
    #              [5.,0]])
    batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
    eps = 1e-6
    probability = tf.clip_by_value(y_pred, eps, 1-eps)
    cost_values = tf.math.log(1-probability)*batch_cost_matrix
    loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
    return loss

In [291]:
def train_model_no_shuffle(model,feature,y,binary=True,file=None):
    if binary:
        ind1 = ((y[0]==0)|(y[0]==1)|(y[0]==2)|(y[0]==3)|(y[0]==4)|(y[0]==6))
        ind2 = ((y[1]==0)|(y[1]==1)|(y[1]==2)|(y[1]==3)|(y[1]==4)|(y[1]==6))
        ind3 = ((y[2]==0)|(y[2]==1)|(y[2]==2)|(y[2]==3)|(y[2]==4)|(y[2]==6))
        ind01 = ((y[0]==4)|(y[0]==1)|(y[0]==2)|(y[0]==3)|(y[0]==6))
        ind11 = ((y[1]==4)|(y[1]==1)|(y[1]==2)|(y[1]==3)|(y[1]==6))
        ind21 = ((y[2]==4)|(y[2]==1)|(y[2]==2)|(y[2]==3)|(y[2]==6))
        
        y_01 = y[0][ind1].copy()
        y_02 = y[1][ind2].copy()
        y_03 = y[2][ind3].copy()

        #ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind01] = 1
        y_02[ind11] = 1
        y_03[ind21] = 1 
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:10}
    else:
        ind1 = ((y[0]==2)|(y[0]==6))
        ind2 = ((y[1]==2)|(y[1]==6))
        ind3 = ((y[2]==2)|(y[2]==6))
        #ind = ((y==2)|(y==6))
        y_01 = y[0][ind1].copy()
        y_02 = y[1][ind2].copy()
        y_03 = y[2][ind3].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:4,1:1,2:1}#{0:2,1:1,2:10,3:2}


    x_train = feature[0][ind1]
    y_train = y_01
    x_valid = feature[1][ind2]
    y_valid = oc.transform(y_02[:,np.newaxis]).toarray()
    x_test = feature[2][ind3]
    y_test = oc.transform(y_03[:,np.newaxis]).toarray()
    
    early_stopping = callbacks.EarlyStopping(patience = 10,
                                             monitor = 'val_loss', 
                                             #baseline = 0.9,
                                             restore_best_weights=True)
    history = model.fit(x_train,y_train,validation_data=(x_valid,y_valid),
                        epochs=300,batch_size=32,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = accuracy_score(np.argmax(y_test,axis=1),y_test,y_pred_t>0.5)
        
        y_pred_v=model.predict(x_valid)
        valid = accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = accuracy_score(np.argmax(y_train,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',confusion_matrix(np.argmax(y_train,axis=1),np.argmax(y_pred_ta,axis=1)))
        print('valid: \n',confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))
        print('test:%f'%test)
        print('valid:%f'%valid)
        print('train:%f'%train)

    else:
        y_pred_t=model.predict(x_test)
        test = accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        y_pred_v=model.predict(x_valid)
        valid = accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = accuracy_score(np.argmax(y_train,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',confusion_matrix(np.argmax(y_train,axis=1),np.argmax(y_pred_ta,axis=1)))
        print('valid: \n',confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        print('test:%f'%test)
        print('valid:%f'%valid)
        print('train:%f'%train)
    if file != None:
        model.save_model(file)
    #return train,test    

## ANN

In [ ]:
ind = ((y==1)|(y==2)|(y==3)|(y==6))
#ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y[ind].copy()
#y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature.loc[ind,:],y_01,test_size=0.2,random_state=123,shuffle=False)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555,shuffle=True)

In [ ]:
input_ = layers.Input(shape=feature.shape[1:])#feature.shape[1:]
l1 = layers.Dense(128,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(input_)
#drop1 = layers.Dropout(0.2)(l1)
bn1 = layers.BatchNormalization()(l1)
l2 = layers.Dense(64,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(bn1)
#drop2 = layers.Dropout(0.2)(l2)
bn2 = layers.BatchNormalization()(l2)
l3 = layers.Dense(32,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(bn2)
#drop3 = layers.Dropout(0.2)(l3)
bn3 = layers.BatchNormalization()(l3)
l4 = layers.Dense(16,activation='elu',
                 #kernel_regularizer = regularizers.l2(0.001),
                 )(bn3)
#drop4 = layers.Dropout(0.2)(l4)
#l5 = layers.Dense(8,activation='relu')(drop4)
#drop5 = layers.Dropout(0.5)(l5)
output = layers.Dense(3,activation='softmax')(l4)
model = Model(inputs=[input_],outputs=[output])

In [292]:
reg = regularizers.l2(0)

model = models.Sequential()
#model.add(layers.BatchNormalization())
model.add(layers.Dense(128,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  kernel_regularizer = reg,
                       #use_bias=False
                 ))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(64,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  kernel_regularizer = reg,
                      # use_bias=False
                 ))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(32,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  kernel_regularizer = reg,
                       #use_bias=False
                 ))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(16,#activation='elu',
                 kernel_regularizer = reg,
                       #use_bias=False
                 ))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(2,activation='softmax'))

In [293]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train model

In [294]:
train_model_no_shuffle(model,(feature,feature2,feature3),
                       (np.array(Y),np.array(Y2),np.array(Y3)),
                       False)
player.Beep(1000,1000)

Epoch 1/300
14/14 [==============================] - 0s 21ms/step - loss: 0.9231 - accuracy: 0.5045 - val_loss: 0.6614 - val_accuracy: 0.5210
Epoch 2/300
14/14 [==============================] - 0s 5ms/step - loss: 0.6147 - accuracy: 0.6674 - val_loss: 0.6102 - val_accuracy: 0.6218
Epoch 3/300
14/14 [==============================] - 0s 5ms/step - loss: 0.5109 - accuracy: 0.7602 - val_loss: 0.5545 - val_accuracy: 0.7563
Epoch 4/300
14/14 [==============================] - 0s 5ms/step - loss: 0.4155 - accuracy: 0.8439 - val_loss: 0.5043 - val_accuracy: 0.8151
Epoch 5/300
14/14 [==============================] - 0s 5ms/step - loss: 0.3907 - accuracy: 0.8529 - val_loss: 0.4296 - val_accuracy: 0.8403
Epoch 6/300
14/14 [==============================] - 0s 5ms/step - loss: 0.3288 - accuracy: 0.8846 - val_loss: 0.3619 - val_accuracy: 0.8655
Epoch 7/300
14/14 [==============================] - 0s 5ms/step - loss: 0.2816 - accuracy: 0.9276 - val_loss: 0.3152 - val_accuracy: 0.8908
Epoch 8/300


1

In [ ]:
#sc = StandardScaler(with_mean=True)
#feature_sc = sc.fit_transform(feature)
#feature2_sc = sc.transform(feature2)
#pca = PCA(n_components=160,copy=True)
#feature_pca = pca.fit_transform(feature)
#feature2_pca = pca.transform(feature2)
train,test,sc,pca = train_model(model,feature,np.array(Y),False)
#rest = test_model(model,feature2,np.array(y2),sc)
#acc = [train,valid,test,rest]

In [194]:
test_model(model,np.array(feature2),np.array(Y2),sc,pca,False)

acc:0.476773
[[  0  88   0]
 [  0 195   0]
 [  0 126   0]]


0.4767726161369193

In [92]:
ind = ((y2==1)|(y2==2)|(y2==6))
oc = OneHotEncoder()
y_01 = oc.fit_transform(y2[ind,np.newaxis]).toarray()
y_pred=model.predict(sc.transform(np.array(feature2)[ind,:]))
ind2=np.where(np.argmax(y_pred,axis=1) != np.argmax(y_01,axis=1))
#ind2

In [51]:
model.save('./model/ann2.h5')

In [43]:
ind = np.array((y==0)|(y==1)|(y==2)|(y==6))
#feature
ind

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [ ]:
#test_model(model,feature2_sc,np.array(y2))
ind = ((y==1)|(y==2)|(y==6))
y_01 = y[ind].copy()
oc = OneHotEncoder()
y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
pred=model.predict(feature_sc[ind,:])
np.argmax(pred,axis=1)
np.argmax(y_01,axis=1)
metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(pred,axis=1))

In [ ]:
#acc_ann = pd.DataFrame(acc,columns=['dnn1'],index=['train','valid','test','rest data'])
#acc_ann_com=pd.concat([acc_ann_com,acc_ann.T])
#acc_ann.loc['drop_mDWT',:]=[train,valid,test,rest]
acc_ann_com.loc['dnn2',:]=acc
acc_ann_com

# test feature from Left or Right

In [ ]:
input_ = layers.Input(shape=feature.loc[:,ind_temp].shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
#l3 = layers.Dense(32,activation='elu')(drop2)
#drop3 = layers.Dropout(0.2)(l3)
#l4 = layers.Dense(16,activation='selu')(l3)
#drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop2)
model = Model(inputs=[input_],outputs=[output])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [209]:
acc={}
cols = ['LEFT','RIGHT']

#sc = StandardScaler(with_mean=True)

for col in cols:
    ind_temp=feature.columns.str.contains(col)
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    #acc[col] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 23125 samples, validate on 5782 samples
Epoch 1/200
23125/23125 [==============================] - 4s 167us/sample - loss: 1.9657 - accuracy: 0.5308 - val_loss: 1.3537 - val_accuracy: 0.6404
Epoch 2/200
23125/23125 [==============================] - 3s 109us/sample - loss: 1.3913 - accuracy: 0.6479 - val_loss: 1.1555 - val_accuracy: 0.7008
Epoch 3/200
23125/23125 [==============================] - 3s 110us/sample - loss: 1.2045 - accuracy: 0.7052 - val_loss: 1.0439 - val_accuracy: 0.7771
Epoch 4/200
23125/23125 [==============================] - 2s 105us/sample - loss: 1.1076 - accuracy: 0.7394 - val_loss: 0.9467 - val_accuracy: 0.7608
Epoch 5/200
23125/23125 [==============================] - 2s 86us/sample - loss: 1.0526 - accuracy: 0.7501 - val_loss: 0.9212 - val_accuracy: 0.7508
Epoch 6/200
23125/23125 [==============================] - 2s 93us/sample - loss: 0.9990 - accuracy: 0.7677 - val_loss: 0.8937 - val_accuracy: 0.7975
Epoch 7/200
23125/23125 [======================

23125/23125 [==============================] - 2s 71us/sample - loss: 0.8517 - accuracy: 0.8045 - val_loss: 0.8904 - val_accuracy: 0.8210
Epoch 9/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.8091 - accuracy: 0.8131 - val_loss: 1.0212 - val_accuracy: 0.8426
Epoch 10/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7750 - accuracy: 0.8260 - val_loss: 1.0699 - val_accuracy: 0.8438
Epoch 11/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7591 - accuracy: 0.8365 - val_loss: 0.9963 - val_accuracy: 0.8513
Epoch 12/200
23125/23125 [==============================] - 2s 70us/sample - loss: 0.7071 - accuracy: 0.8416 - val_loss: 1.0759 - val_accuracy: 0.8464
Epoch 13/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.6955 - accuracy: 0.8444 - val_loss: 1.0100 - val_accuracy: 0.8442
Epoch 14/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.7017 - accuracy: 0.8478

In [ ]:
acc_lr=pd.DataFrame(acc,index=['train','valid','test','rest data'])#.to_csv('./results/acc_lr_ann.csv')
#acc_com=pd.concat([acc_ann_com.drop(['dnn','dnn1'],'index'),acc_lr.T])
acc_com

# test features from 2 of 8 signals

In [ ]:
#test features from 2 of 8 signals

acc_2f={}
cols = [ 'LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF',
        'RIGHT_TA','RIGHT_TS','RIGHT_BF', 'RIGHT_RF']


for p in combinations(cols[:4],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)
    
for p in combinations(cols[4:],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)

In [ ]:
#acc_2f_ann = pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T
#acc_com = pd.concat([acc_com,acc_2f_ann])
acc_com.to_csv('./results/dropna/acc_com_ann.csv')

In [ ]:
pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T

# Test on some of rest

In [ ]:
some_of_rest = ['正常/P940_MSham_B_Walking_trial_6_emg.csv',
                '正常/P940_M050_B_Walking_trial_4_emg.csv',
                '正常/P812_M100_A_Walking_trial_3_emg.csv',
                '正常/P645_M050_A_Walking_trial_3_emg.csv',
                '正常/P623_Msham_B_Walking_trial_2_emg.csv',
                '正常/P551_M50_B_Walking_trial_6_emg.csv',
                'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
                'P551_M050_2_B_FoG_trial_2_emg.csv']
#booster = xgb.Booster()
#booster.load_model('./model/XGBoost_W256_S64_Left.json')
#model = xgb.XGBClassifier()
#model._Booster = booster
acc = []
columns=['LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF']
for file in some_of_rest:
    path = './data/'+file
    feature2,y2 = dp.pipeline_feature(path,width=256,stride=64,scaler=False,
                                      threshold_WAMP=threshold_WAMP,
                                      threshold_ZC=threshold_ZC,
                                      threshold_SSC=threshold_SSC,
                                      bins=bins,
                                      ranges=HIST_range,
                                      show_para=False,
                                      filt = 250)
    feature2_sc = sc.transform(feature2)
    acc += [test_model(model,feature2_sc,y2)]
print(acc)

In [ ]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

## RNN

In [92]:
files = np.array(df.columns)
X = []
Y = []
#sc = StandardScaler(with_mean=False)
i = 0
start = time.time()
for file in files:
    emg_data = pd.read_csv('./data/'+file)
    #emg_data = emg_data.fillna({'LEFT_TA':emg_data.LEFT_TA.mean(),
    #                            'LEFT_TS':emg_data.LEFT_TS.mean(),
    #                            'LEFT_BF':emg_data.LEFT_BF.mean(),
    #                            'LEFT_RF':emg_data.LEFT_RF.mean(),
    #                            'RIGHT_TA':emg_data.RIGHT_TA.mean(),
    #                            'RIGHT_TS':emg_data.RIGHT_TS.mean(),
    #                            'RIGHT_BF':emg_data.RIGHT_BF.mean(),
    #                            'RIGHT_RF':emg_data.RIGHT_RF.mean()})
    #emg_data = emg_data[emg_data.Label1 == emg_data.Label2].reset_index(drop=True)
    emg_data = emg_data.dropna().reset_index(drop=True)
    x = np.array(emg_data.iloc[:,3:])
    y = np.array(emg_data.Label2)
    #x,y = dp.generate_window_slide_data(emg_data,width=width,stride=stride,scaler=False,same_label=True)
    X += x.tolist()
    Y += y.tolist()
    i += 1
    print('i:%d'%i,x.shape)
end = time.time()
duration = end-start
print('Duration: %f'%(duration))

i:1 (16583, 8)
i:2 (4489, 8)
i:3 (35121, 8)
i:4 (31027, 8)
i:5 (30783, 8)
i:6 (159539, 8)
i:7 (225455, 8)
i:8 (362273, 8)
i:9 (28868, 8)
i:10 (20878, 8)
i:11 (25322, 8)
i:12 (24242, 8)
i:13 (21728, 8)
i:14 (27916, 8)
i:15 (27146, 8)
i:16 (25947, 8)
i:17 (26203, 8)
i:18 (89500, 8)
i:19 (70282, 8)
i:20 (79595, 8)
i:21 (73280, 8)
Duration: 4.539805


In [10]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y.copy()
        #ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-1')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    x_full = np.reshape(x_full,(-1,128*8))
    sc = StandardScaler(with_mean=False)
    #sc = MinMaxScaler((0,1))
    x_train = sc.fit_transform(x_full)
    x_train = x_train.reshape((-1,128,8))
    #pca = PCA(n_components=150)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = np.reshape(x_test,(-1,128*8))
    x_test = sc.transform(x_test)
    x_test = x_test.reshape((-1,128,8))
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 10,
                                             monitor = 'val_accuracy', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=[x_test,y_test],
                        epochs=300,batch_size=500,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc#,pca

In [ ]:
feature_re = feature_sc.reshape((-1,X.shape[1],1))
x_full,x_test,y_full,y_test = train_test_split(np.array(feature_re),np.array(y_01),test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [11]:
input_ = layers.Input(shape=[128,8])
lstm1 = layers.LSTM(50,return_sequences=True)(input_)
drop1 = layers.Dropout(0.2)(lstm1)
lstm2 = layers.LSTM(50,return_sequences=True)(drop1)
drop2 = layers.Dropout(0.2)(lstm2)
#lstm3 = layers.LSTM(50,return_sequences=True)(drop2)
#drop3 = layers.Dropout(0.2)(lstm3)
lstm4 = layers.LSTM(50)(drop2)
drop4 = layers.Dropout(0.2)(lstm4)
dense1 = layers.Dense(64,activation='relu')(drop4)
drop5 = layers.Dropout(0.2)(dense1)
dense2 = layers.Dense(32,activation='relu')(drop5)
output = layers.Dense(3,activation='softmax')(dense2)
model = Model(inputs=[input_],outputs=[output])

In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=500,
                   callbacks=[early_stopping])

In [13]:
train,test,sc = train_model(model,np.array(x),np.array(y),False)

Train on 8872 samples, validate on 2219 samples
Epoch 1/300
8872/8872 [==============================] - 88s 10ms/sample - loss: 0.9841 - accuracy: 0.4876 - val_loss: 0.8857 - val_accuracy: 0.5002
Epoch 2/300
8872/8872 [==============================] - 82s 9ms/sample - loss: 0.8572 - accuracy: 0.5571 - val_loss: 0.8288 - val_accuracy: 0.6471
Epoch 3/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.8051 - accuracy: 0.6443 - val_loss: 0.7573 - val_accuracy: 0.6670
Epoch 4/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.7433 - accuracy: 0.6677 - val_loss: 0.7142 - val_accuracy: 0.6791
Epoch 5/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.7212 - accuracy: 0.6787 - val_loss: 0.6996 - val_accuracy: 0.6886
Epoch 6/300
8872/8872 [==============================] - 87s 10ms/sample - loss: 0.7071 - accuracy: 0.6878 - val_loss: 0.6952 - val_accuracy: 0.7008
Epoch 7/300
8872/8872 [==============================] - 81s 9

KeyboardInterrupt: 

In [ ]:
feature2_re = feature2_sc.reshape((-1,feature.shape[1],1))

In [ ]:
test_model(model,x_test,y_test)

In [113]:
cost_matrix=np.array([[0,1,1,1],
                  [1,0,1,1],
                  [10,100,0,10],
                  [1.,1.,1,0]])
cost_matrix=np.array([[0,2.],
                  [5.,0]])

In [58]:
P0 = 55080/(55080+8181)
P1 = 8181/(55080+8181)
P = [P0,P1]
cost_vec = np.zeros(2)
for i in range(2):
    for j in range(2):
        if i == j:
            continue
        cost_vec[i]+=1/(1-P[i])*P[j]*cost_matrix[i,j]

In [59]:
cost_vec

array([2., 5.])

In [112]:
tf.transpose(t)

<tf.Tensor: id=1567100, shape=(4, 4), dtype=float64, numpy=
array([[  0.,   1.,  10.,   1.],
       [  1.,   0., 100.,   1.],
       [  1.,   1.,   0.,   1.],
       [  1.,   1.,  10.,   0.]])>

In [116]:
np.reshape(cost_matrix,(-1,1))

array([[0.],
       [2.],
       [5.],
       [0.]])